<a href="https://colab.research.google.com/github/lengochai97/thesis/blob/master/notebooks/feature_construction/51_User_Activeness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Connect to Google Drive

In [0]:
%%capture

import google.colab.drive

google.colab.drive.mount('/content/gdrive', force_remount=True)

# Install Spark and dependencies

In [0]:
import os

os.environ['HADOOP_VERSION'] = '2.7'
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'
os.environ['SPARK_HOME'] = '/opt/spark'
os.environ['SPARK_VERSION'] = '2.4.3'

In [0]:
%%capture

!wget -qN https://archive.apache.org/dist/spark/spark-$SPARK_VERSION/spark-$SPARK_VERSION-bin-hadoop$HADOOP_VERSION.tgz
!tar -xzf spark-$SPARK_VERSION-bin-hadoop$HADOOP_VERSION.tgz -C /opt
!rm spark-$SPARK_VERSION-bin-hadoop$HADOOP_VERSION.tgz
!rm -rf /opt/spark
!ln -s /opt/spark-$SPARK_VERSION-bin-hadoop$HADOOP_VERSION /opt/spark
!pip install -q findspark

# Create SparkSession

In [0]:
import findspark

findspark.init()

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local[*]').getOrCreate()

# Read files

In [0]:
import json

import pyspark.sql.functions as F
import pyspark.sql.types as T

In [0]:
DATA_PATH = '/content/gdrive/My Drive/dataset/adressa/one_week'

In [0]:
with open(os.path.join(DATA_PATH, 'schema', 'clean.json')) as file:
  clean_schema = T.StructType.fromJson(json.load(file))

In [0]:
df_clean = spark.read.json(os.path.join(DATA_PATH, 'clean'), schema=clean_schema)

# Calculate user activeness

In [0]:
from pyspark.sql import Window

In [0]:
import math

@F.udf(returnType=T.DoubleType())
def calculate_user_activeness(time_list):
  user_activeness = 0.5
  if len(time_list) < 2:
    return user_activeness
  for i in range(1, len(time_list)):
    dt = time_list[i] - time_list[i-1]
    st = math.exp(-1.2e-05 * dt)
    user_activeness = min(1., user_activeness * st + 0.32)
  return user_activeness

In [0]:
df_user_activeness = (
    df_clean
    .select(
        F.column('userId'),
        F.column('time'),
    )
    .withColumn(
        'timeList',
        F.collect_list(F.column('time')).over(
            Window
            .partitionBy('userId')
            .orderBy('time')
            .rowsBetween(Window.unboundedPreceding, Window.currentRow)
        ),
    )
    .withColumn(
        'userActiveness',
        calculate_user_activeness(F.column('timeList')),
    )
    .select(
        F.column('userId'),
        F.column('time'),
        F.column('userActiveness'),
    )
)

# Write files

In [0]:
df_user_activeness = (
    df_user_activeness
    .repartition(1)
    .sortWithinPartitions(
        F.column('time'),
        F.column('userId'),
    )
)

In [0]:
%%time

df_user_activeness.write.json(os.path.join(DATA_PATH, 'user_activeness'))

CPU times: user 33 ms, sys: 10.8 ms, total: 43.7 ms
Wall time: 1min 15s


In [0]:
with open(os.path.join(DATA_PATH, 'schema', 'user_activeness.json'), 'w+') as file:
  json.dump(df_user_activeness.schema.jsonValue(), file)